In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when,col,sum

spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

transactions = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "transactions") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

transactions.show()

+--------------+------+----------------+
|transaction_id|amount|transaction_date|
+--------------+------+----------------+
|             1|   150|      2024-07-01|
|             2|   200|      2024-07-01|
|             3|    75|      2024-07-01|
|             4|   300|      2024-07-02|
|             5|    50|      2024-07-02|
|             6|   120|      2024-07-03|
+--------------+------+----------------+



In [5]:
transactions\
    .withColumn("even_amount",when(col("amount")%2==0,col("amount")))\
    .withColumn("odd_amount",when(col("amount")%2!=0,col("amount")))\
    .groupBy(col("transaction_date"))\
    .agg(sum("odd_amount").alias("odd_sum"),
         sum("even_amount").alias("even_sum"))\
    .show()

+----------------+-------+--------+
|transaction_date|odd_sum|even_sum|
+----------------+-------+--------+
|      2024-07-02|   NULL|     350|
|      2024-07-03|   NULL|     120|
|      2024-07-01|     75|     350|
+----------------+-------+--------+

